<a href="https://colab.research.google.com/github/TofunmiSodimu/Novelty-Detection/blob/main/CorruptTestingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
This code gets all movable/grabbable objs, their locations at certain times of the day,
and the activities being performed at that time from the HOMER_PLUS dataset.
"""

'\nThis code gets all movable/grabbable objs, their locations at certain times of the day,\nand the activities being performed at that time from the HOMER_PLUS dataset.\n'

In [ ]:
# Download HOMER_PLUS dataset from github
!wget https://github.com/TofunmiSodimu/Novelty-Detection/zipball/master.zip


In [ ]:
# Unzip folder
!unzip /content/master.zip

In [14]:
import random
import numpy as np
import csv

class novelty:
  def __init__(self):
    # Create time_arr and room_id arr
    self.room_ids = ['1','41','162','241']
    self.room_dict = {'1':'bathroom','41':'dining_room','162':'bedroom','241':'home_office'}
    self.data = []
    self.corrupt_dets = []
    self.corrupt_percent = [1, 5, 10, 20]

  def corrupt_data(self):
    # For each csv file in testing_data file
    for num in range(24):

      # Reset everything
      self.data = []
      self.corrupt_dets = []

      # Open CSV file and get data
      number = str(num).zfill(3)
      file_name = '/content/TofunmiSodimu-Novelty-Detection-b0e7828/Uncorrupt_Testing_Data_RoomsOnly/test' + number + '.csv'

      with open(file_name,"r") as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
          self.data.append(row)

      # Randomly corrupt each day by percentages in self.corrupt_percent
      for percent in self.corrupt_percent:
        self.corrupt_data_percent(percent,number)

  def corrupt_data_percent(self,percent,number):

      # Randomly pick instances from the data for corruption
      indices_chosen = np.random.choice(len(self.data),int(0.01*percent*len(self.data)),False)

      # Randomly change the room location of these randomly chosen instances
      for idx in indices_chosen:
        value = {'Index':idx,'Room_Og':self.data[idx]['Room_ID']}
        while value['Room_Og'] == self.data[idx]['Room_ID']:
          self.data[idx]['Room_ID'] = random.choice(self.room_ids)
        self.data[idx]['Room_name'] = self.room_dict[self.data[idx]['Room_ID']]
        value['Room_Changed'] = self.data[idx]['Room_ID']
        self.data[idx]['Anomaly_Label'] = 1

        # Store corrupting details
        self.corrupt_dets.append(value)

      # Save corrupted data as csv
      self.convert_csv(number,percent,'test')

      # Save corruption details as csv
      self.convert_csv_corrupt(number,percent,'corrupt_dets')

  def convert_csv(self,number,percent,name):
    field_names = ['Time', 'Obj_ID', 'Obj_name', 'Room_ID', 'Room_name', 'Activity', 'Anomaly_Label']
    name = name + number + '_' + str(percent) + 'percent.csv'
    with open(name,'w') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=field_names)
      writer.writeheader()
      writer.writerows(self.data)

  def convert_csv_corrupt(self,number,percent,name):
    field_names = ['Index', 'Room_Og', 'Room_Changed']
    name = name + number + '_' + str(percent) + 'percent.csv'
    with open(name,'w') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=field_names)
      writer.writeheader()
      writer.writerows(self.corrupt_dets)


In [15]:
novelty().corrupt_data()